In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import spacy
import wordcloud
import os # navigating your computer's files
import sys
pd.options.mode.chained_assignment = None #suppress warnings

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
nltk.download('wordnet')
nltk.download('punkt')

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python -m spacy download en_core_web_md
import en_core_web_md
text_to_nlp = spacy.load('en_core_web_md')

!wget -q --show-progress "https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%201%20-%205/Session%203%20-%20NLP/yelp_final.csv"
yelp_full = pd.read_csv('yelp_final.csv')

#adds 2 new columns:stars and text
needed_columns = ["stars","text"]
yelp = yelp_full[needed_columns]
yelp.head()

#prints all 5 star reviews
num_stars =  5#@param {type:"integer"}
for t in yelp[yelp['stars'] == num_stars]['text'].head(20).values:
    print(t)

#creates column identifying whether the review is good or bad
def is_good_review(num_stars):
    if num_stars >= 4:
        return True
    else:
        return False

# Change the stars column to either be 'good' or 'bad'.
yelp['is_good_review'] = yelp['stars'].apply(is_good_review)
yelp.head()

X_text = yelp['text']
y = yelp['is_good_review']

#tokenizes reviews, removes stopwords, prepares vocab
def tokenize(text):
    clean_tokens = []
    for token in text_to_nlp(text):
        if (not token.is_stop) & (token.lemma_ != '-PRON-') & (not token.is_punct):
            clean_tokens.append(token.lemma_)
    return clean_tokens
bow_transformer = CountVectorizer(analyzer=tokenize, max_features=800).fit(X_text)
X = bow_transformer.transform(X_text)
pd.DataFrame(X.toarray())

#splits training and testing data; training to train log regression model, testing to test model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

#fit data into model
logistic_model = LogisticRegression()
logistic_model.fit(X_train,y_train)

#test model
logistic_model = LogisticRegression()
logistic_model.fit(X_train,y_train)
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
print (accuracy)

#sets weight of each review, whether good or bad
def get_word_weight(word,model,vocab_dict):
  if word in vocab_dict:
    weight = model.coef_[0][vocab_dict[word]]
    return weight
  else:
    return "That word is not in our Vocabulary!"

#import model
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()

#fit model, 80% accuracy
nb_model.fit(X_train,y_train)
y_pred = nb_model.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
print (accuracy)

#find word similarity to increase accuracy of model
text_to_nlp = en_core_web_md.load()
doc = text_to_nlp(u"pineapple and textbook")
word1 = doc[0]
word2 = doc[2]
word1.similarity(word2)

#remove stop words, pronouns, and punctuation, and convert each word to a spaCy object
def tokenize_vecs(text):
    clean_tokens = []
    for token in text_to_nlp(text):
        if (not token.is_stop) & (token.lemma_ != '-PRON-') & (not token.is_punct):
            clean_tokens.append(token)
    return clean_tokens

X_word2vec = []
for text in X_text:
  review = tokenize_vecs(text) # returns cleaned list of spacy tokens
  review_vec = [0]*300
  for word in review:
    review_vec += word.vector
  review_vec = review_vec / len(review)
  X_word2vec.append(review_vec)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2024-01-27 16:26:26.920216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-27 16:26:26.920270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-27 16:26:26.921090: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-27 16:26:27.818052: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
yelp_final.csv      100%[===================>] 743.14K  --.

In [ ]:
w2v_model = LogisticRegression(max_iter = 1000)
# train-test split
X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(X_word2vec, y, test_size=0.2, random_state=101)
w2v_model.fit(X_train_word2vec, y_train_word2vec)
w2v_preds = w2v_model.predict(X_test_word2vec)
accuracy = accuracy_score(y_test_word2vec, w2v_preds)
print (accuracy)
# prints the accuracy of the model by testing it on the second half of the training data

0.755
